# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Wolf  ->  D. J. Wolf  |  ['J. Wolf']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Li  ->  J. Li  |  ['J. Li']
M. Demianenko  ->  M. Demianenko  |  ['M. Demianenko']
J.-U. Pott  ->  J.-U. Pott  |  ['J.-U. Pott']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


N. Martin  ->  N. Martin  |  ['N. Martin']
J. Shi  ->  J. Shi  |  ['J. Shi']
Arxiv has 67 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.03872


extracting tarball to tmp_2505.03872...

 done.


J. Wolf  ->  D. J. Wolf  |  ['J. Wolf']


...

Found 95 bibliographic references in tmp_2505.03890/aanda.bbl.
Issues with the citations
syntax error in line 466: '=' expected
Retrieving document from  https://arxiv.org/e-print/2505.04275
extracting tarball to tmp_2505.04275...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.04475
extracting tarball to tmp_2505.04475... done.


Found 177 bibliographic references in tmp_2505.04475/Main_Paper.bbl.
Retrieving document from  https://arxiv.org/e-print/2505.04610


extracting tarball to tmp_2505.04610...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.03872-b31b1b.svg)](https://arxiv.org/abs/2505.03872) | **The eROSITA Final Equatorial Depth Survey (eFEDS): SDSS spectroscopic observations of X-ray sources**  |
|| C. Aydar, et al. -- incl., <mark>J. Wolf</mark> |
|*Appeared on*| *2025-05-08*|
|*Comments*| **|
|**Abstract**|            We present one of the largest uniform optical spectroscopic surveys of X-ray selected sources to date that were observed as a pilot study for the Black Hole Mapper (BHM) survey. The BHM program of the Sloan Digital Sky Survey (SDSS)-V is designed to provide optical spectra for hundreds of thousands of X-ray selected sources from the SRG/eROSITA all-sky survey. This significantly improves our ability to classify and characterise the physical properties of large statistical populations of X-ray emitting objects. Our sample consists of 13079 sources in the eROSITA eFEDS performance verification field, 12011 of which provide reliable redshifts from 0<z<5.8. The vast majority of these objects were detected as point-like sources (X-ray flux limit F(0.5-2 keV)>6.5x10^-15 erg/s/cm^2) and were observed for about 20 years with fibre-fed SDSS spectrographs. After including all available redshift information for the eFEDS sources from the dedicated SDSS-V plate programme and archival data, we visually inspected the SDSS optical spectra to verify the reliability of these redshift measurements and the performance of the SDSS pipeline. The visual inspection allowed us to recover reliable redshifts (for 99% of the spectra with a signal-to-noise ratio of >2) and to assign classes to the sources, and we confirm that the vast majority of our sample consists of active galactic nuclei (AGNs). Only ~3% of the eFEDS/SDSS sources are Galactic objects. We also show the diversity of the optical spectra of the X-ray selected AGNs and provide spectral stacks with a high signal-to-noise ratio in various sub-samples with different redshift and optical broad-band colours. Our AGN sample contains optical spectra of (broad-line) quasars, narrow-line galaxies, and optically passive galaxies. It is considerably diverse in its colours and in its levels of nuclear obscuration.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.03876-b31b1b.svg)](https://arxiv.org/abs/2505.03876) | **SHELLQs-JWST Unveils the Host Galaxies of Twelve Quasars at $z>6$**  |
|| X. Ding, et al. -- incl., <mark>K. Jahnke</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2025-05-08*|
|*Comments*| *22 pages, 10 figures, 6 tables. submitted to ApJ; Comments welcome*|
|**Abstract**|            The advent of JWST has opened new horizons in the study of quasar host galaxies during the reionization epoch ($z > 6$). Building upon our previous initial uncovering of stellar light from two quasar host galaxies at these redshifts, we now report the detection of the stellar light from the full Cycle 1 sample of 12 distant moderate-luminosity quasar ($M_{1450}>-24$ mag) host galaxies at $z>6$ from the Hyper Suprime-Cam Subaru Strategic Program (HSC-SSP). Using JWST/NIRCam observations at 1.5 and 3.6 $\mu$m combined with 2D image decomposition analysis, we successfully detect the host galaxies in 11 out of 12 targets, underscoring the high detection rates achievable with moderate-luminosity quasars. Based on two-band photometry and SED fitting, we find that our host galaxies are massive with log~M$_*$/M$_{\odot} = 9.5\text{-}11.0$. The effective radii range from 0.6 to 3.2 kpc which is comparable to the sizes of inactive galaxies with similar masses as measured with imaging from COSMOS-Web. Intriguingly, the two quasar hosts with post-starburst features, which reside at the high-mass end of our sample and exhibit relatively compact morphologies, are consistent (at the 1-$\sigma$ level) with quiescent galaxies at $z\sim4\text{-}5$ in terms of their sizes and stellar mass surface densities. These findings support the so-called galaxy compaction scenario already being valid at the reionization epoch ($z>6$), in which gas inflows during starburst phases drive centrally concentrated star formation followed by rapid quenching, bridging the structural transition of massive galaxies from relatively extended star-forming disks to compact quiescent systems.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.03890-b31b1b.svg)](https://arxiv.org/abs/2505.03890) | **Broad line region echo from rapidly accreting intermediate-mass black hole candidate SDSS J144850.08+160803.1. The first probe of intra-night variability and reverberation mapping**  |
|| <mark>M. Demianenko</mark>, et al. -- incl., <mark>J.-U. Pott</mark> |
|*Appeared on*| *2025-05-08*|
|*Comments*| *Submitted to A&A on 14.04.2025, 18 pages, 13 figures*|
|**Abstract**|            Context. Elusive intermediate-mass black holes (IMBHs; $100~M_{\odot} \leq M_{\rm BH}\leq 2\times10^{5}~M_{\odot}$) can act as ``time-squeezing'' machines, enabling studies of AGN geometry through reverberation mapping (RM) on much shorter timescales than their supermassive siblings. Aims. The broad line region (BLR) radius constraints for IMBH candidates with different Eddington ratios probe the unexplored faint end of radius--luminosity ($R-L$) relations and allow us to build a robust $M_{\rm BH}$ estimator. This study aims to (a) confirm a rapidly accreting IMBH candidate; (b) demonstrate the feasibility of the first photometric BLR reverberation mapping (RM) campaign for this class of objects. Methods. The IMBH candidate J1448+16 was identified in the broad H$\alpha$-selected spectroscopic sample from SDSS. We carried out narrow-band H$\alpha$ and broad-band SDSS~g\'~monitoring to check small-scale variability and extract the time lag between the BLR and accretion disk (AD) continuum during 5 months (March--July 2024) using a 60-cm telescope at the Caucasus Mountain Observatory. Results. We confirmed the candidate as an active IMBH using XMM-Newton as a bright X-ray point source with the photon index $\Gamma = 2.32^{+0.15}_{-0.13}$, suggesting the high accretion rate. From a combination of SDSS optical spectra and XMM-Newton observations, we estimate the black hole mass in the range $\sim 0.9-2.4~\times10^{5} M_{\odot}$ and the Eddington rate from $\sim 37-112\%$. We report high-amplitude $\sim 60\%$ intra-night ($\sim 2$~h) H$\alpha$ variability in this highly accreting IMBH. In addition, we extracted a tentative measurement of a BLR RM radius $\sim 1-8~\mathrm{days}$. Conclusions. This work is a proof of concept for further IMBH variability studies and BLR RM campaigns, which will be essential for the calibration of the $R-L$ relation at the faint end.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.04475-b31b1b.svg)](https://arxiv.org/abs/2505.04475) | **Mass Modeling the Andromeda Dwarf Galaxies: Andromeda VI and Andromeda XXIII**  |
|| C. S. Pickett, et al. -- incl., <mark>N. Martin</mark> |
|*Appeared on*| *2025-05-08*|
|*Comments*| *19 pages, 16 figures. Submitted to MNRAS for review*|
|**Abstract**|            Accurately mapping the mass profiles of low mass dwarf spheroidal (dSph) galaxies allows us to test predictions made by dark matter (DM) models. To date, such analyses have primarily been performed on Milky Way (MW) satellites. Meanwhile, the Andromeda Galaxy (M31) is home to 35 known dwarf galaxies, yet only two have been successfully mass-modeled so far. In order to better understand the nature of dark matter, a more comprehensive study of Local Group dwarfs is necessary. In this study, we have undertaken a dynamical study of two higher-luminosity Andromeda dwarf galaxies: Andromeda VI (And VI) and Andromeda XXIII (And XXIII). We infer an enclosed mass for And VI of M(r < r$_{\rm{h}}$) = (4.9 $\pm$ 1.5) $\times$ 10$^{7}$ M$_{\odot}$, corresponding to a mass-to-light ratio of $[M/L]_{r_{\rm{h}}}$ = (27.1 $\pm$ 8.2) M$_{\odot}$/L$_{\odot}$. We infer an enclosed mass for And XXIII of M(r < r$_{\rm{h}}$) = (3.1 $\pm$ 1.9) $\times$ 10$^{7}$ M$_{\odot}$, corresponding to a mass-to-light ratio of $[M/L]_{\rm{r_{h}}}$ = (90.2 $\pm$ 53.9) M$_{\odot}$/L$_{\odot}$. Using the dynamical Jeans modeling tool, GravSphere, we determine And VI and And XXIII's dark matter density at 150 pc, finding $\rho_{\rm{DM,VI}}$(150 pc) = (1.4 $\pm$ 0.5) $\times$ 10$^{8}$ M$_{\odot}$ kpc$^{-3}$ and $\rho_{\rm{DM,XXIII}}$(150 pc) = 0.5$\substack{+0.4 \\ -0.3} \times$ 10$^{8}$ M$_{\odot}$ kpc$^{-3}$. Our results make And VI the first mass-modeled M31 satellite to fall into the cuspy regime, while And XXIII has a lower density, implying either a more cored central dark matter density, or a lowering of the density through tides. This adds And XXIII to a growing list of M31 dwarfs with a central density lower than most MW dwarfs and lower than expected for isolated dwarfs in the Standard Cosmology. This could be explained by the M31 dwarfs having experienced stronger tides than their Milky Way counterparts.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.04275-b31b1b.svg)](https://arxiv.org/abs/2505.04275) | **J-PAS and PFS surveys in the era of dark energy and neutrino mass measurements**  |
|| F. Qin, <mark>Y. Wang</mark>, G.-B. Zhao |
|*Appeared on*| *2025-05-08*|
|*Comments*| *17 pages, 11 figures, 6 tables; Forecast code is publicly available at this https URL*|
|**Abstract**|            Fisher-matrix forecasts are presented for the cosmological surveys of the Javalambre Physics of the Accelerating Universe Astrophysical Survey (J-PAS) and the Subaru Prime Focus Spectrograph (PFS). The wide, low-redshift coverage of J-PAS and the high-density, high-redshift mapping of PFS are strongly complementary: combining the two reduces marginalized uncertainties on all primary parameters compared with either survey individually. Adding the joint J-PAS+PFS data to next-generation CMB measurements from CMB-S4 and \textsc{LiteBird} yields an expected precision of $\sigma(\sum m_\nu)=0.017\,$eV in the $\Lambda$CDM$+\sum m_\nu+N_{\rm eff}$ framework, sufficient to disfavour the inverted neutrino hierarchy at $2.35\,\sigma$ if the true mass sum equals the normal-ordering minimum. Motivated by recent DESI results, we also forecast within a $w_0w_a$CDM$+\sum m_\nu+N_{\rm eff}$ cosmology, adopting the DESI\,DR2 best-fit values ($w_0=-0.758$, $w_a=-0.82$) as fiducial. The combination CMB+J-PAS+PFS then delivers $\sigma(w_0)=0.044$ and $\sigma(w_a)=0.18$, corresponding to a $5.1\,\sigma$ preference for a time-varying dark-energy equation of state. These findings show that J-PAS and PFS, especially when coupled with Stage-IV CMB observations, will provide competitive tests of neutrino physics and the dynamics of cosmic acceleration.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.04610-b31b1b.svg)](https://arxiv.org/abs/2505.04610) | **Early and Extensive Ultraviolet Through Near Infrared Observations of the Intermediate-Luminosity Type Iax Supernovae 2024pxl**  |
|| W. B. Hoogendam, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-05-08*|
|*Comments*| *18 pages, 12 figures, to be submitted to AAS journals*|
|**Abstract**|            We present ultraviolet (UV) through near-infrared (NIR) photometric and spectroscopic observations of the nearby SN 2024pxl, the third Type Ia supernova (SN Ia) in NGC 6384. SN 2024pxl is a Type Iax supernova (SN Iax) with an intermediate luminosity ($M_r = -16.99\pm0.32$ mag) and an average SN Iax light curve decline rate. SN 2024pxl was discovered $\sim$3 days after first light, and the rising light curve follows a single power law that is inconsistent with significant interaction with a companion star or circumstellar material. Our extensive NIR photometric coverage is comparable to that of the well-observed SNe Iax 2005hk and 2012Z, and we demonstrate that the $J-H$ colors of SNe Iax differ from normal SNe Ia and appear to be more homogeneous as a class. Spectroscopically, we report the earliest-ever NIR spectrum of a SN Iax as measured from maximum light ($t\approx-9$ days): a featureless continuum with similarities to a $\sim$9,000 K blackbody, and the line velocities are consistent with a mixed-ejecta structure, with C, Si, and Fe having similar velocities and velocity evolutions. We find a tentative correlation between the $H$-band break Co II velocity $\sim$20 days post-peak and absolute magnitude, with more luminous SNe Iax showing faster Co II velocities. Our observations suggest that SN 2024pxl resulted from the thermonuclear disruption of a CO white dwarf star that undergoes deflagration burning.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures []
exported in  _build/html/2505.03872.md
found figures ['tmp_2505.03876/./figure4_a.png', 'tmp_2505.03876/./figure4_b.png', 'tmp_2505.03876/./figure1_F356W.png', 'tmp_2505.03876/./figure_ratio_M1450.png']
copying  tmp_2505.03876/./figure4_a.png to _build/html/
copying  tmp_2505.03876/./figure4_b.png to _build/html/
copying  tmp_2505.03876/./figure1_F356W.png to _build/html/
copying  tmp_2505.03876/./figure_ratio_M1450.png to _build/html/
exported in  _build/html/2505.03876.md
    + _build/html/tmp_2505.03876/./figure4_a.png
    + _build/html/tmp_2505.03876/./figure4_b.png
    + _build/html/tmp_2505.03876/./figure1_F356W.png
    + _build/html/tmp_2505.03876/./figure_ratio_M1450.png
found figures ['tmp_2505.03890/./images/ICCF_hyperparams_cccd_tresh_0.6_sig_0.4_11042025_10000_randomseedsame_agnfirst_upd2_VERBOSETRUE_oldval.png', 'tmp_2505.03890/./images/field.png', 'tmp_2505.03890/./images/se_img_for_paper_rm-1.png']
copying  tmp_2505.03890/./images/ICCF_hyperparams_cccd_t

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\etal}{{\it et al.}\xspace}$
$\newcommand{\erosita}{{eROSITA}\xspace}$
$\newcommand{\srg}{\HB{\em SRG}\xspace}$
$\newcommand{\xmm}{\HB{\em XMM-Newton}\xspace}$
$\newcommand{\rosat}{\HB{\em ROSAT}\xspace}$
$\newcommand{\chandra}{\HB{\em Chandra}\xspace}$
$\newcommand{\gaia}{\HB{\em Gaia}\xspace}$
$\newcommand{\HB}[1]{\textcolor{red}{#1}}$
$\newcommand{\TL}[1]{\textcolor{purple}{#1}}$</div>



<div id="title">

# The eROSITA Final Equatorial Depth Survey (eFEDS):: SDSS spectroscopic observations of X-ray sources

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.03872-b31b1b.svg)](https://arxiv.org/abs/2505.03872)<mark>Appeared on: 2025-05-08</mark> - 

</div>
<div id="authors">

C. Aydar, et al. -- incl., <mark>J. Wolf</mark>

</div>
<div id="abstract">

**Abstract:** We present one of the largest uniform optical spectroscopic surveys of X-ray selected sources to date that were observed as a pilot study for the Black Hole Mapper (BHM) survey.  The BHM program of the Sloan Digital Sky Survey (SDSS)-V is designed to provide optical spectra for hundreds of thousands of X-ray selected sources from the SRG/eROSITA all-sky survey.  This significantly improves our ability to classify and characterise the physical properties of large statistical populations of X-ray emitting objects.  Our sample consists of 13 079 sources in the eROSITA eFEDS performance verification field, 12 011 of which provide reliable redshifts from $0\lesssim z\leq 5.8$ .  The vast majority of these objects were detected as point-like sources (X-ray flux limit $F_{\rm 0.5-2 keV}\gtrsim 6.5\times 10^{-15}$ erg/s/cm $^2$ ) and were observed for about 20 years with fibre-fed SDSS spectrographs.  After including all available redshift information for the eFEDS sources from the dedicated SDSS-V plate programme and archival data, we visually inspected the SDSS optical spectra to verify the reliability of these redshift measurements and the performance of the SDSS pipeline.  The visual inspection allowed us to recover reliable redshifts (for 99 \% of the spectra with a signal-to-noise ratio of $>2$ ) and to assign classes to the sources, and we confirm that the vast majority of our sample consists of active galactic nuclei (AGNs).  Only $\sim3\%$ of the eFEDS/SDSS sources are Galactic objects.  We analysed the completeness and purity of the spectroscopic redshift catalogue, in which the spectroscopic completeness increases from $48\%$ (full sample) to $81\%$ for a cleaner, brighter ( $r_{\rm AB}<21.38$ ) sample that we defined by considering a high X-ray detection likelihood, a reliable counterpart association, and an optimal sky coverage.  We also show the diversity of the optical spectra of the X-ray selected AGNs and provide spectral stacks with a high signal-to-noise ratio in various sub-samples with different redshift and optical broad-band colours.  Our AGN sample contains optical spectra of (broad-line) quasars, narrow-line galaxies, and optically passive galaxies.  It is considerably diverse in its colours and in its levels of nuclear obscuration.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.03872"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\TBD}{\textcolor{blue}{\textbf{TBD}}}$
$\newcommand{\todo}[1]{\textcolor{blue}{\textbf{TODO: #1}}}$
$\newcommand{\blue}[1]{\textcolor{black}{#1}}$
$\newcommand{\newblue}[1]{\textcolor{blue}{#1}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# SHELLQs-JWST Unveils the Host Galaxies of Twelve Quasars at $z>6$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.03876-b31b1b.svg)](https://arxiv.org/abs/2505.03876)<mark>Appeared on: 2025-05-08</mark> -  _22 pages, 10 figures, 6 tables. submitted to ApJ; Comments welcome_

</div>
<div id="authors">

X. Ding, et al. -- incl., <mark>K. Jahnke</mark>, <mark>J. Li</mark>

</div>
<div id="abstract">

**Abstract:** The advent of JWST has opened new horizons in the study of quasar host galaxies during the reionization epoch ( $z > 6$ ). Building upon our previous initial uncovering of stellar light from two quasar host galaxies at these redshifts, we now report the detection of the stellar light from the full Cycle 1 sample of 12 distant moderate-luminosity quasar ( $M_{1450}>-24$ mag) host galaxies at $z>6$ from the Hyper Suprime-Cam Subaru Strategic Program (HSC-SSP). Using JWST/NIRCam observations at 1.5 and 3.6 $\mu$ m combined with 2D image decomposition analysis, we successfully detect the host galaxies in 11 out of 12 targets, underscoring the high detection rates achievable with moderate-luminosity quasars. Based on two-band photometry and SED fitting, we find that our host galaxies are massive with log M $_*$ /M $_{\odot} = 9.5 \text{--} 11.0$ . The effective radii range from 0.6 to 3.2 kpc which is comparable to the sizes of inactive galaxies with similar masses as measured with imaging from COSMOS-Web.Intriguingly, the two quasar hosts with post-starburst features, which reside at the high-mass end of our sample and exhibit relatively compact morphologies, are consistent (at the 1- $\sigma$ level) with quiescent galaxies at $z\sim\text{4--5}$ in terms of their sizes and stellar mass surface densities.These findings support the so-called galaxy compaction scenario already being valid at the reionization epoch ( $z>6$ ), in which gas inflows during starburst phases drive centrally concentrated star formation followed by rapid quenching, bridging the structural transition of massive galaxies from relatively extended star-forming disks to compact quiescent systems.

</div>

<div id="div_fig1">

<img src="tmp_2505.03876/./figure4_a.png" alt="Fig7.1" width="50%"/><img src="tmp_2505.03876/./figure4_b.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** (Left) Size - stellar mass relation of our quasar host galaxies compared with the control sample of star-forming galaxies (SFGs) and from COSMOS-Web \citep[see][]{Yang2025arXiv250407185Y} at redshift $6<z<7$. The two gray stars with arrows are the host measurements with size or stellar mass as upper limits. We also show the quiescent galaxies (QGs) sample at $4<z<5$ and the best-fit result from the COSMOS-Web.
We measure the sizes of our sample host galaxies using the F356W filter, while the COSMOS-Web sample adopts the F444W filter (see Section \ref{subsec:SizeMass} for methodological details).
The results show that the sizes of our quasar host sample appear to be similar to the SFGs for a given stellar mass, while the two post-starbursts are consistent with the QGs within 1-$\sigma$ level. The dashed horizontal line indicates the PSF size at redshift 6.
(Right) Projected 2D stellar mass surface density within the \reff compared with those of quiescent galaxies from the literature  ([Skelton, Whitaker and Momcheva 2014](), [Carnall, McLure and Dunlop 2023](), [Vanzella, Claeyssens and Welch 2023](), [Setton and Brammer 2024](), [Setton, Khullar and Miller 2024](), [Weibel and Setton 2024]())  as well as with $1<z<2.5$ AGN host galaxies revealed by COSMOS-web  ([Tanaka, Silverman and Ding 2025]()) .
Most of our quasar hosts fall below the typical quiescent threshold \citep[i.e., dashed line as introduced in][]{Weibel2024}. The two post-starburst targets are in the top region of our sample. The typical uncertainty associated with the density measurement of our sample is 0.5 dex. (*fig:size-mass*)

</div>
<div id="div_fig2">

<img src="tmp_2505.03876/./figure1_F356W.png" alt="Fig3" width="100%"/>

**Figure 3. -** Quasar-host decomposition of NIRCam/F356W images for all twelve targets. The rows are as follows from top to bottom: (1) original science image (data), (2) data minus quasar model, i.e., the host galaxy and neighbors, (3) best-fit host \sersic model(s), and (4) Signal-to-noise maps of the host significance; the noise is a combination of that from the data and the scatter of the corresponding top-five PSFs. In Row (4), we show the elliptical apertures, within which we calculate the host signal-to-noise ratio (SNR), as reported in Table \ref{tab:F356W_result}. When drawing these apertures, we use solid lines to highlight those with confirmed host detections. The inferred host-to-total flux ratios from the best fit are also indicated in the bottom panels. Our simulation results indicate that even in challenging cases, such as J0844$-$0132, we are able to accurately recover the host galaxy properties, as demonstrated in Figure \ref{fig:sim_onecase}.
     (*fig:F356Wfit*)

</div>
<div id="div_fig3">

<img src="tmp_2505.03876/./figure_ratio_M1450.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
Host-to-total flux ratio as a function of the quasar's absolute magnitude. The plot includes three luminous $z\sim6$ quasars reported in [Yue, Eilers and Simcoe (2024)](). Our results demonstrate that the intrinsically low luminosity of our quasars leads to a significantly higher host-to-total flux ratio than found in luminous quasars. Note that the photometric measurements of the host galaxies in [Yue, Eilers and Simcoe (2024)]() are subject to minimal statistical uncertainty, resulting in exceptionally small error bars for their sample.
 (*fig:ratio*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.03876"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$</div>



<div id="title">

# Broad line region echo from rapidly accreting intermediate-mass black hole candidate SDSS J144850.08+160803.1: The first probe of intra-night variability and reverberation mapping

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.03890-b31b1b.svg)](https://arxiv.org/abs/2505.03890)<mark>Appeared on: 2025-05-08</mark> -  _Submitted to A&A on 14.04.2025, 18 pages, 13 figures_

</div>
<div id="authors">

<mark>M. Demianenko</mark>, et al. -- incl., <mark>J.-U. Pott</mark>

</div>
<div id="abstract">

**Abstract:** Elusive intermediate-mass black holes (IMBHs; $100 M_{\odot} \leq M_{\rm BH}\leq 2\times10^{5} M_{\odot}$ ) can act as "time-squeezing" machines, enabling studies of AGN geometry through reverberation mapping (RM) on much shorter timescales than their supermassive siblings. The broad line region (BLR) radius constraints for IMBH candidates with different Eddington ratios probe the unexplored faint end of radius--luminosity ( $R-L$ ) relations and allow us to build a robust $M_{\rm BH}$ estimator. This study aims to (a) confirm a rapidly accreting IMBH candidate; (b) demonstrate the feasibility of the first photometric BLR reverberation mapping (RM) campaign for this class of objects. The IMBH candidate J1448+16 was identified in the broad H $\alpha$ -selected spectroscopic sample from SDSS. We carried out narrow-band H $\alpha$ and broad-band SDSS g \' monitoring to check small-scale variability and extract the time lag between the BLR and accretion disk (AD) continuum  during 5 months (March--July 2024) using a 60-cm telescope at the Caucasus Mountain Observatory. We confirmed the candidate as an active IMBH using XMM-Newton as a bright X-ray point source with the photon index $\Gamma = 2.32^{+0.15}_{-0.13}$ , suggesting the high accretion rate. From a combination of SDSS optical spectra and XMM-Newton observations, we estimate the black hole mass in the range $\sim 0.9-2.4 \times10^{5} M_{\odot}$ and the Eddington rate from $\sim 37-112\%$ .    We report high-amplitude $\sim 60\%$ intra-night ( $\sim 2$ h) H $\alpha$ variability in this highly accreting IMBH. In addition, we extracted a tentative measurement of a BLR RM radius $\sim 1-8 \mathrm{days}$ . This work is a proof of concept for further IMBH variability studies and BLR RM campaigns, which will be essential for the calibration of the $R-L$ relation at the faint end.

</div>

<div id="div_fig1">

<img src="tmp_2505.03890/./images/ICCF_hyperparams_cccd_tresh_0.6_sig_0.4_11042025_10000_randomseedsame_agnfirst_upd2_VERBOSETRUE_oldval.png" alt="Fig3" width="100%"/>

**Figure 3. -** Time delay inference dependence on interpolation step of ICCF for whole light curve.
Color code: blue -- validation star, red -- AGN. Points without error bars correspond to the flat ICCF, where CCCD failed. The CCCD was run with the following ICCF parameters ({\sc nsim}: 5000, {\sc sigmode}: 0.4, {\sc thres}: 0.6, {\sc nbin}: 100). The Monte Carlo sampling slightly depends on the random seed, but for interpolation steps less than one day always shows smaller time lags with smaller uncertainties for AGN compared to the validation star.
 (*fig:iccf_hyperparams_ccpd*)

</div>
<div id="div_fig2">

<img src="tmp_2505.03890/./images/field.png" alt="Fig5" width="100%"/>

**Figure 5. -** An example of an RC600 g\' field (FoV 22.7 arcmin) with reference stars marked with magenta circles and validation stars with dark cyan circles, indicating a S/N ratio near each star. The circle diameter is 6" for display purposes (the photometry aperture diameter is 3").
 (*fig:foV*)

</div>
<div id="div_fig3">

<img src="tmp_2505.03890/./images/se_img_for_paper_rm-1.png" alt="Fig6" width="100%"/>

**Figure 6. -** General pipeline of deriving $\mathrm{H\beta}$ \citep{2013ApJ...767..149B} BLR radius in this Appendix.  (*fig: r_blr_from_xray*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.03890"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\LCDM}{\LambdaCDM}$
$\newcommand{\kms}{kms^{-1}}$
$\newcommand{\gravsphere}{\texttt{GravSphere}}$
$\newcommand{\DMVI}{\rho_{\rm{DM,VI}}(150 pc) = (1.4 \pm 0.5) \times 10^{8} M_{\odot} kpc^{-3}}$
$\newcommand{\DMXXIII}{\rho_{\rm{DM,XXIII}}(150 pc) = 0.5\substack{+0.4 \ -0.3} \times 10^{8} M_{\odot} kpc^{-3}}$
$\newcommand{\rhVI}{r_{\rm{h,VI}} = (489\pm{22}) pc}$
$\newcommand{\rhXXIII}{r_{\rm{h,XXIII}} = 1170 \substack{+95 \ -94} pc}$
$\newcommand{\MLVI}{(27.1 \pm 8.2) M_{\odot}/L_{\odot}}$
$\newcommand{\MLXXIII}{(90.2 \pm 53.9) M_{\odot}/L_{\odot}}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Mass Modeling the Andromeda Dwarf Galaxies: Andromeda VI and Andromeda XXIII

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.04475-b31b1b.svg)](https://arxiv.org/abs/2505.04475)<mark>Appeared on: 2025-05-08</mark> -  _19 pages, 16 figures. Submitted to MNRAS for review_

</div>
<div id="authors">

C. S. Pickett, et al. -- incl., <mark>N. Martin</mark>

</div>
<div id="abstract">

**Abstract:** Accurately mapping the mass profiles of low mass dwarf spheroidal (dSph) galaxies allows us to test predictions made by dark matter (DM) models. To date, such analyses have primarily been performed on Milky Way (MW) satellites. Meanwhile, the Andromeda Galaxy (M31) is home to 35 known dwarf galaxies, yet only two have been successfully mass-modeled so far. In order to better understand the nature of dark matter, a more comprehensive study of Local Group dwarfs is necessary. In this study, we have undertaken a dynamical study of two higher-luminosity Andromeda dwarf galaxies: Andromeda VI (And VI) and Andromeda XXIII (And XXIII). We infer an enclosed mass for And VI of  M(r $<$ r $_{\rm{h}}$ ) = (4.9 $\pm$ 1.5) $\times$ 10 $^{7}$ M $_{\odot}$ , corresponding to a mass-to-light ratio of $[M/L]_{r_{\rm{h}}}$ = $\MLVI$ . We infer an enclosed mass for And XXIII of  M(r $<$ r $_{\rm{h}}$ ) = (3.1 $\pm$ 1.9) $\times$ 10 $^{7}$ M $_{\odot}$ , corresponding to a mass-to-light ratio of $[M/L]_{r_{\rm{h}}}$ = $\MLXXIII$ . Using the dynamical Jeans modeling tool, $\gravsphere$ , we determine And VI and And XXIII's dark matter density at 150 pc, finding $\DMVI$ and $\DMXXIII$ . Our results make And VI the first mass-modeled M31 satellite to fall into the cuspy regime, while And XXIII has a lower density, implying either a more cored central dark matter density, or a lowering of the density through tides. As And XXIII was quenched early in its evolution, dark matter core formation from repeated inflow/outflow due to gas cooling and stellar feedback is disfavored. This adds And XXIII to a growing list of M31 dwarfs with a central density lower than most MW dwarfs and lower than expected for isolated dwarfs in the Standard Cosmology. This could be explained by the M31 dwarfs having experienced stronger tides than their Milky Way counterparts.

</div>

<div id="div_fig1">

<img src="tmp_2505.04475/./figures/And6_rho.png" alt="Fig7" width="100%"/>

**Figure 7. -** Andromeda VI Dark matter density profiles determined by $\gravsphere$ (solid black line), with 1 and 2$\sigma$ being denoted by the dark gray and light gray shaded regions, respectively. The pink shaded region is the density profile created via $\langle{\text{SFR}}\rangle$ abundance matching. The blue shaded region is the density profile created via $M_{\rm{*}}$ abundance matching. The blue vertical line is the measured $r_{\rm{h}}$ of And VI. The pink dashed line is at 150 pc, or the distance at which DM density is measured. (*fig:And6_SFR_Den*)

</div>
<div id="div_fig2">

<img src="tmp_2505.04475/./figures/And6AllPlot.png" alt="Fig2" width="100%"/>

**Figure 2. -** **Top Left**: Histogram of number of stars split into 40 velocity bins of 15 $\kms$ each. The light gray bars represent the non-member velocities while the dark gray bars represent member star velocities. The vertical dashed black bar is the systemic velocity of And VI from C13. **Bottom Left**: Velocity versus radial distance from measured center of And VI. The points are shaded depending on their respective probabilities. Each point's uncertainty is too small to be seen relative to each point but is adequately represented by the size of each marker. The dashed light gray horizontal line is the measured half-light radius of And VI from [ and Savino (2022)](https://ui.adsabs.harvard.edu/abs/2022ApJ...938..101S). **Right**: CMD of the RGB overdensity of And VI. The light gray points are non-member stars, as determined by our probability calculations. The colored points represent member stars, shaded according to their respective probability. The RGB is overlaid with the best-fit-by-eye isochrone of parameters [Fe/H] = -1.5 dex, [$\alpha$/Fe] = 0, and age = 10 Gyr. (*fig:And6all*)

</div>
<div id="div_fig3">

<img src="tmp_2505.04475/./figures/And23_rho.png" alt="Fig8" width="100%"/>

**Figure 8. -** Same as Figure \ref{fig:And6_SFR_Den} but for Andromeda XXIII. (*fig:And23_SFR_Den*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.04475"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

480  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
